In [30]:
import pandas as pd

In [31]:
import seaborn as sns
df=sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [33]:
df.sex.value_counts(),df.smoker.value_counts(),df.day.value_counts(),df.time.value_counts()

(sex
 Male      157
 Female     87
 Name: count, dtype: int64,
 smoker
 No     151
 Yes     93
 Name: count, dtype: int64,
 day
 Sat     87
 Sun     76
 Thur    62
 Fri     19
 Name: count, dtype: int64,
 time
 Dinner    176
 Lunch      68
 Name: count, dtype: int64)

## Feature Encoding

In [34]:
df.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [35]:
df.time.unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [36]:
x=df.drop(columns=['tip'])
y=df['tip']

In [37]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)

In [38]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [39]:
cat_column=['sex', 'smoker','time']
for i in cat_column:
  x_train[i]=le.fit_transform(x_train[i])
  x_test[i]=le.transform(x_test[i])

In [40]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
num_column = ['total_bill', 'size']
for i in num_column:

    x_train_num = x_train[[i]].values.reshape(-1, 1)
    x_test_num = x_test[[i]].values.reshape(-1, 1)


    x_train[[i]] = ss.fit_transform(x_train_num)
    x_test[[i]] = ss.transform(x_test_num)

In [44]:
x_train=pd.get_dummies(x_train,columns=['day'],drop_first=True)
x_test=pd.get_dummies(x_test,columns=['day'],drop_first=True)

In [45]:
x_train.head(),x_test.head()

(     total_bill  sex  smoker  time      size  day_Fri  day_Sat  day_Sun
 228   -0.793062    1       0     0 -0.612141    False     True    False
 208    0.463227    1       1     0 -0.612141    False     True    False
 96     0.807307    1       1     0 -0.612141     True    False    False
 167    1.313709    1       0     0  1.519421    False    False     True
 84    -0.484419    1       0     1 -0.612141    False    False    False,
      total_bill  sex  smoker  time      size  day_Fri  day_Sat  day_Sun
 24    -0.045461    1       0     0 -0.612141    False     True    False
 6     -1.308609    1       0     0 -0.612141    False    False     True
 153    0.495235    1       0     0  1.519421    False    False     True
 211    0.648413    1       1     0  1.519421    False     True    False
 198   -0.825069    0       1     1 -0.612141    False    False    False)

##Model Training

In [46]:
from sklearn.svm import SVR
svm=SVR()
svm.fit(x_train,y_train)


SVR()

In [47]:
y_pred=svm.predict(x_test)

In [50]:
from sklearn.metrics import r2_score, mean_absolute_percentage_error
r2_score(y_test,y_pred),mean_absolute_percentage_error(y_test,y_pred)

(0.44015453383165426, 0.27453239106752053)

In [55]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'kernel': ['linear', 'rbf'],  # Test different kernels
    'C': [0.1, 1, 10, 100],       # Regularization parameter
    'epsilon': [0.1, 0.2, 0.5]   # Epsilon in the epsilon-SVR model
}

grid_search = GridSearchCV(
    SVR(),
    param_grid=param_grid,
    scoring='neg_mean_squared_error',  # Use a scoring metric (e.g., MSE)
    cv=5,                              # 5-fold cross-validation
    verbose=1                          # Display progress
)

grid_search.fit(x_train, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Best Parameters: {'C': 1, 'epsilon': 0.5, 'kernel': 'linear'}
Best Score: -1.2208973069128284


In [56]:
grid_prediction=grid_search.predict(x_test)

In [57]:
r2_score(y_test,grid_prediction),mean_absolute_percentage_error(y_test,grid_prediction)

(0.44583767630104554, 0.2768830217739322)